In [1]:
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler, OneHotEncoderEstimator, StringIndexer
from pyspark.sql import SparkSession, DataFrame, functions as F
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline, Transformer
from pyspark.ml.util import DefaultParamsReadable, DefaultParamsWritable, DefaultParamsReader 
from pyspark.ml import PipelineModel
from pyspark.ml.param.shared import HasOutputCols, Param, Params, TypeConverters
from pyspark import keyword_only

In [2]:
spark.conf.set(
  "fs.azure.account.key.sparkmltrainig.blob.core.windows.net",
  "<key>")

In [3]:
data = spark.read.csv("wasbs://data@sparkmltrainig.blob.core.windows.net/train.csv", inferSchema = True, header = True)

train_df, test_df = data.randomSplit([0.9, 0.1], seed=42)
print(train_df.count())
print(test_df.count())


800
91

In [4]:
train_df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
PassengerId|Survived|Pclass| Name| Sex| Age|SibSp|Parch| Ticket| Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
 1| 0| 3|Braund, Mr. Owen ...| male|22.0| 1| 0| A/5 21171| 7.25| null| S|
 2| 1| 1|Cumings, Mrs. Joh...|female|38.0| 1| 0| PC 17599|71.2833| C85| C|
 3| 1| 3|Heikkinen, Miss. ...|female|26.0| 0| 0|STON/O2. 3101282| 7.925| null| S|
 4| 1| 1|Futrelle, Mrs. Ja...|female|35.0| 1| 0| 113803| 53.1| C123| S|
 6| 0| 3| Moran, Mr. James| male|null| 0| 0| 330877| 8.4583| null| Q|
 7| 0| 1|McCarthy, Mr. Tim...| male|54.0| 0| 0| 17463|51.8625| E46| S|
 8| 0| 3|Palsson, Master. ...| male| 2.0| 3| 1| 349909| 21.075| null| S|
 10| 1| 2|Nasser, Mrs. Nich...|female|14.0| 1| 0| 237736|30.0708| null| C|
 11| 1| 3|Sandstrom, Miss. ...|female| 4.0| 1| 1| PP 9549| 16.7| G6| S|
 12| 1| 1|Bonnell, Miss. El...|female|58.0| 0| 0| 113783| 26.55| C103| S|
 13| 0| 3|Saundercock, Mr. ...| male|20.0| 0| 0| A/5. 2151| 8.05| null| S|
 14| 0| 3|Andersson, Mr. An...| male|39.0| 1| 5| 347082| 31.275| null| S|
 15| 0| 3|Vestrom, Miss. Hu...|female|14.0| 0| 0| 350406| 7.8542| null| S|
 16| 1| 2|Hewlett, Mrs. (Ma...|female|55.0| 0| 0| 248706| 16.0| null| S|
 17| 0| 3|Rice, Master. Eugene| male| 2.0| 4| 1| 382652| 29.125| null| Q|
 18| 1| 2|Williams, Mr. Cha...| male|null| 0| 0| 244373| 13.0| null| S|
 19| 0| 3|Vander Planke, Mr...|female|31.0| 1| 0| 345763| 18.0| null| S|
 20| 1| 3|Masselmani, Mrs. ...|female|null| 0| 0| 2649| 7.225| null| C|
 21| 0| 2|Fynney, Mr. Joseph J| male|35.0| 0| 0| 239865| 26.0| null| S|
 22| 1| 2|Beesley, Mr. Lawr...| male|34.0| 0| 0| 248698| 13.0| D56| S|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
only showing top 20 rows

In [5]:
train_df.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in train_df.columns]).show()
test_df.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in test_df.columns]).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
 0| 0| 0| 0| 0|162| 0| 0| 0| 0| 618| 2|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
 0| 0| 0| 0| 0| 15| 0| 0| 0| 0| 69| 0|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+

In [6]:
def evaluate_initials(df: DataFrame) -> DataFrame:
  dizip_initials = {k:v for k,v in (zip(['Mlle','Mme','Ms','Dr', 'Major','Lady','Countess', 'Jonkheer','Col','Rev', 'Capt','Sir','Don'], ['Miss','Miss','Miss', 'Mr','Mr','Mrs','Mrs', 'Other','Other','Other', 'Mr','Mr','Mr']))}
  _df = df.withColumn('Initial',  F.regexp_extract( df['Name'], ('([A-Za-z]+)\.'),1 ) )
  _df = _df.replace(dizip_initials,1,'Initial')
  return _df
  
def handle_missing_age(df: DataFrame) -> DataFrame:
    _df = df
    _df = _df.withColumn('Age', 
           F.when((F.isnull(_df['Age'])) & (_df['Initial'] == 'Mr') , 33 )\
            .otherwise(F.when((F.isnull(_df['Age'])) 
                              & (_df['Initial'] == 'Mrs') , 36)\
            .otherwise(F.when((F.isnull(_df['Age'])) 
                              & (_df['Initial'] == 'Master') , 5)\
            .otherwise(F.when((F.isnull(_df['Age'])) 
                              & (_df['Initial'] == 'Miss') , 22)\
            .otherwise(F.when((F.isnull(_df['Age'])) 
                              & (_df['Initial'] == 'Other') , 46)\
            .otherwise(_df['Age']) )))))
    return _df

def create_family_size(df: DataFrame) -> DataFrame:
  _df = df.withColumn('FamilySize', df['Parch'] + df['SibSp'] + 1 )
  return _df

def create_is_alone(df: DataFrame) -> DataFrame:
  _df = df.withColumn('IsAlone', F.when(df['FamilySize'] > 1, 0).otherwise(1))
  return _df

def create_fare_per_person(df: DataFrame) -> DataFrame:  
  _df = df.withColumn('FarePerPerson', df['Fare'] /df['FamilySize'])
  return _df

def drop_rows_with_null(df: DataFrame, col) -> DataFrame:
  _df = df = df.filter(df[col].isNotNull())
  return _df

def change_to_index(df: DataFrame, col) -> DataFrame:
  indexer = StringIndexer(inputCol=col, outputCol='{0}_indexed'.format(col))
  _df = indexer.fit(df).transform(df)
  return _df

def change_to_one_hot_encoded(df: DataFrame, cols) -> DataFrame:
  for col in cols:
    df = change_to_index(df, col)
  col_indexed = ['{0}_indexed'.format(col) for col in cols]
  col_encoded = ['{0}_encoded'.format(col) for col in cols]
  encoder = OneHotEncoderEstimator(inputCols=col_indexed, outputCols=col_encoded)
  _df = encoder.fit(df).transform(df)
  return _df

<>:3: DeprecationWarning: invalid escape sequence \.
<>:3: DeprecationWarning: invalid escape sequence \.
<command-2671224804740862>:3: DeprecationWarning: invalid escape sequence \.
 _df = df.withColumn('Initial', F.regexp_extract( df['Name'], ('([A-Za-z]+)\.'),1 ) )

In [7]:
train_df = drop_rows_with_null(handle_missing_age(evaluate_initials(create_fare_per_person(create_is_alone(create_family_size(train_df))))), 'Embarked')
test_df = handle_missing_age(evaluate_initials(create_fare_per_person(create_is_alone(create_family_size(test_df)))))

In [8]:
train_df = change_to_one_hot_encoded(train_df, ['Sex', 'Initial', 'Embarked'])
test_df = change_to_one_hot_encoded(test_df, ['Sex', 'Initial', 'Embarked'])


In [9]:
cols_to_drop=['PassengerId', 'Cabin', 'Ticket', 'Name', 'Sex', 'Initial', 'Embarked']
train_df = train_df.drop(*cols_to_drop)
test_df = test_df.drop(*cols_to_drop)

In [10]:
train_df.show()

+--------+------+----+-----+-----+-------+----------+-------+------------------+-----------+---------------+----------------+-------------+---------------+----------------+
Survived|Pclass| Age|SibSp|Parch| Fare|FamilySize|IsAlone| FarePerPerson|Sex_indexed|Initial_indexed|Embarked_indexed| Sex_encoded|Initial_encoded|Embarked_encoded|
+--------+------+----+-----+-----+-------+----------+-------+------------------+-----------+---------------+----------------+-------------+---------------+----------------+
 0| 3|22.0| 1| 0| 7.25| 2| 0| 3.625| 0.0| 0.0| 0.0|(1,[0],[1.0])| (4,[0],[1.0])| (2,[0],[1.0])|
 1| 1|38.0| 1| 0|71.2833| 2| 0| 35.64165| 1.0| 2.0| 1.0| (1,[],[])| (4,[2],[1.0])| (2,[1],[1.0])|
 1| 1|35.0| 1| 0| 53.1| 2| 0| 26.55| 1.0| 2.0| 0.0| (1,[],[])| (4,[2],[1.0])| (2,[0],[1.0])|
 0| 3|35.0| 0| 0| 8.05| 1| 1| 8.05| 0.0| 0.0| 0.0|(1,[0],[1.0])| (4,[0],[1.0])| (2,[0],[1.0])|
 0| 3|33.0| 0| 0| 8.4583| 1| 1| 8.4583| 0.0| 0.0| 2.0|(1,[0],[1.0])| (4,[0],[1.0])| (2,[],[])|
 0| 1|54.0| 0| 0|51.8625| 1| 1| 51.8625| 0.0| 0.0| 0.0|(1,[0],[1.0])| (4,[0],[1.0])| (2,[0],[1.0])|
 0| 3| 2.0| 3| 1| 21.075| 5| 0| 4.215| 0.0| 3.0| 0.0|(1,[0],[1.0])| (4,[3],[1.0])| (2,[0],[1.0])|
 1| 3|27.0| 0| 2|11.1333| 3| 0| 3.7111| 1.0| 2.0| 0.0| (1,[],[])| (4,[2],[1.0])| (2,[0],[1.0])|
 1| 2|14.0| 1| 0|30.0708| 2| 0| 15.0354| 1.0| 2.0| 1.0| (1,[],[])| (4,[2],[1.0])| (2,[1],[1.0])|
 1| 3| 4.0| 1| 1| 16.7| 3| 0| 5.566666666666666| 1.0| 1.0| 0.0| (1,[],[])| (4,[1],[1.0])| (2,[0],[1.0])|
 1| 1|58.0| 0| 0| 26.55| 1| 1| 26.55| 1.0| 1.0| 0.0| (1,[],[])| (4,[1],[1.0])| (2,[0],[1.0])|
 0| 3|20.0| 0| 0| 8.05| 1| 1| 8.05| 0.0| 0.0| 0.0|(1,[0],[1.0])| (4,[0],[1.0])| (2,[0],[1.0])|
 0| 3|39.0| 1| 5| 31.275| 7| 0|4.4678571428571425| 0.0| 0.0| 0.0|(1,[0],[1.0])| (4,[0],[1.0])| (2,[0],[1.0])|
 0| 3|14.0| 0| 0| 7.8542| 1| 1| 7.8542| 1.0| 1.0| 0.0| (1,[],[])| (4,[1],[1.0])| (2,[0],[1.0])|
 1| 2|55.0| 0| 0| 16.0| 1| 1| 16.0| 1.0| 2.0| 0.0| (1,[],[])| (4,[2],[1.0])| (2,[0],[1.0])|
 1| 2|33.0| 0| 0| 13.0| 1| 1| 13.0| 0.0| 0.0| 0.0|(1,[0],[1.0])| (4,[0],[1.0])| (2,[0],[1.0])|
 0| 3|31.0| 1| 0| 18.0| 2| 0| 9.0| 1.0| 2.0| 0.0| (1,[],[])| (4,[2],[1.0])| (2,[0],[1.0])|
 1| 3|36.0| 0| 0| 7.225| 1| 1| 7.225| 1.0| 2.0| 1.0| (1,[],[])| (4,[2],[1.0])| (2,[1],[1.0])|
 0| 2|35.0| 0| 0| 26.0| 1| 1| 26.0| 0.0| 0.0| 0.0|(1,[0],[1.0])| (4,[0],[1.0])| (2,[0],[1.0])|
 1| 2|34.0| 0| 0| 13.0| 1| 1| 13.0| 0.0| 0.0| 0.0|(1,[0],[1.0])| (4,[0],[1.0])| (2,[0],[1.0])|
+--------+------+----+-----+-----+-------+----------+-------+------------------+-----------+---------------+----------------+-------------+---------------+----------------+
only showing top 20 rows

In [11]:
cols = [c for c in train_df.columns if c != 'Survived']
vec_assembler = VectorAssembler(inputCols=cols, outputCol="features")
vec_train_DF = vec_assembler.transform(train_df)
lr = LinearRegression(featuresCol="features", labelCol="Survived")
lr_model = lr.fit(vec_train_DF)

In [12]:
vec_test_df = vec_assembler.transform(test_df)
pred_df = lr_model.transform(vec_test_df)
pred_df.show()

+--------+------+----+-----+-----+-------+----------+-------+-----------------+-----------+---------------+----------------+-------------+---------------+----------------+--------------------+-------------------+
Survived|Pclass| Age|SibSp|Parch| Fare|FamilySize|IsAlone| FarePerPerson|Sex_indexed|Initial_indexed|Embarked_indexed| Sex_encoded|Initial_encoded|Embarked_encoded| features| prediction|
+--------+------+----+-----+-----+-------+----------+-------+-----------------+-----------+---------------+----------------+-------------+---------------+----------------+--------------------+-------------------+
 1| 3|26.0| 0| 0| 7.925| 1| 1| 7.925| 1.0| 1.0| 0.0| (1,[],[])| (4,[1],[1.0])| (2,[0],[1.0])|(18,[0,1,4,5,6,7,...| 0.5905510743011597|
 0| 3| 2.0| 4| 1| 29.125| 6| 0|4.854166666666667| 0.0| 3.0| 2.0|(1,[0],[1.0])| (4,[3],[1.0])| (2,[],[])|[3.0,2.0,4.0,1.0,...| 0.5026026381839436|
 0| 1|42.0| 1| 0| 52.0| 2| 0| 26.0| 0.0| 0.0| 0.0|(1,[0],[1.0])| (4,[0],[1.0])| (2,[0],[1.0])|(18,[0,1,2,4,5,7,...|0.34593756409002596|
 0| 3| 7.0| 4| 1|39.6875| 6| 0|6.614583333333333| 0.0| 3.0| 0.0|(1,[0],[1.0])| (4,[3],[1.0])| (2,[0],[1.0])|[3.0,7.0,4.0,1.0,...| 0.4099846281289141|
 0| 3|21.0| 0| 0| 7.8| 1| 1| 7.8| 0.0| 0.0| 0.0|(1,[0],[1.0])| (4,[0],[1.0])| (2,[0],[1.0])|(18,[0,1,4,5,6,7,...| 0.1104816241118205|
 0| 2|34.0| 1| 0| 26.0| 2| 0| 13.0| 0.0| 0.0| 0.0|(1,[0],[1.0])| (4,[0],[1.0])| (2,[0],[1.0])|(18,[0,1,2,4,5,7,...|0.20233258877954152|
 0| 3|16.0| 0| 0| 9.2167| 1| 1| 9.2167| 0.0| 0.0| 0.0|(1,[0],[1.0])| (4,[0],[1.0])| (2,[0],[1.0])|(18,[0,1,4,5,6,7,...|0.12901409255927887|
 1| 3|22.0| 0| 0| 7.75| 1| 1| 7.75| 1.0| 1.0| 0.0| (1,[],[])| (4,[1],[1.0])| (2,[0],[1.0])|(18,[0,1,4,5,6,7,...| 0.6049391507353366|
 0| 3|21.0| 0| 0| 7.925| 1| 1| 7.925| 0.0| 0.0| 0.0|(1,[0],[1.0])| (4,[0],[1.0])| (2,[0],[1.0])|(18,[0,1,4,5,6,7,...| 0.110523460678571|
 1| 1|44.0| 0| 0|27.7208| 1| 1| 27.7208| 1.0| 2.0| 1.0| (1,[],[])| (4,[2],[1.0])| (2,[1],[1.0])|[1.0,44.0,0.0,0.0...| 1.0018375122637517|
 1| 3| 5.0| 4| 2|31.3875| 7| 0|4.483928571428572| 1.0| 1.0| 0.0| (1,[],[])| (4,[1],[1.0])| (2,[0],[1.0])|[3.0,5.0,4.0,2.0,...| 0.2912286516482445|
 0| 2|24.0| 0| 0| 10.5| 1| 1| 10.5| 0.0| 0.0| 0.0|(1,[0],[1.0])| (4,[0],[1.0])| (2,[0],[1.0])|(18,[0,1,4,5,6,7,...| 0.260365813786285|
 1| 3|29.0| 0| 2|15.2458| 3| 0|5.081933333333333| 1.0| 2.0| 1.0| (1,[],[])| (4,[2],[1.0])| (2,[1],[1.0])|[3.0,29.0,0.0,2.0...| 0.6513212744347145|
 0| 3|33.0| 0| 0| 7.75| 1| 1| 7.75| 0.0| 0.0| 2.0|(1,[0],[1.0])| (4,[0],[1.0])| (2,[],[])|(18,[0,1,4,5,6,7,...| 0.1479151426729084|
 0| 2|36.0| 0| 0| 10.5| 1| 1| 10.5| 0.0| 0.0| 0.0|(1,[0],[1.0])| (4,[0],[1.0])| (2,[0],[1.0])|(18,[0,1,4,5,6,7,...| 0.2170258709034023|
 1| 3|25.0| 0| 0| 0.0| 1| 1| 0.0| 0.0| 0.0| 0.0|(1,[0],[1.0])| (4,[0],[1.0])| (2,[0],[1.0])|(18,[0,1,5,6,11,1...|0.09342437471895959|
 0| 3| 7.0| 4| 1| 29.125| 6| 0|4.854166666666667| 0.0| 3.0| 2.0|(1,[0],[1.0])| (4,[3],[1.0])| (2,[],[])|[3.0,7.0,4.0,1.0,...| 0.4845443286494092|
 1| 1|26.0| 0| 0| 78.85| 1| 1| 78.85| 1.0| 1.0| 0.0| (1,[],[])| (4,[1],[1.0])| (2,[0],[1.0])|(18,[0,1,4,5,6,7,...| 0.9339201533821605|
 0| 3|24.0| 0| 0| 8.85| 1| 1| 8.85| 1.0| 1.0| 0.0| (1,[],[])| (4,[1],[1.0])| (2,[0],[1.0])|(18,[0,1,4,5,6,7,...| 0.5980839887089275|
 0| 3|22.0| 0| 0| 7.25| 1| 1| 7.25| 0.0| 0.0| 0.0|(1,[0],[1.0])| (4,[0],[1.0])| (2,[0],[1.0])|(18,[0,1,4,5,6,7,...|0.10668588131121126|
+--------+------+----+-----+-----+-------+----------+-------+-----------------+-----------+---------------+----------------+-------------+---------------+----------------+--------------------+-------------------+
only showing top 20 rows

In [13]:
regression_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="Survived", metricName="rmse")

rmse = regression_evaluator.evaluate(pred_df)
print(f"RMSE is {rmse}")

RMSE is 0.34149375724898745

In [14]:
class InitialsTransformer(Transformer, DefaultParamsReadable, DefaultParamsWritable, DefaultParamsReader):
  @keyword_only
  def __init__(self):
    super(InitialsTransformer, self).__init__()

  def _transform(self, df: DataFrame) -> DataFrame:
    dizip_initials = {k:v for k,v in (zip(['Mlle','Mme','Ms','Dr', 'Major','Lady','Countess', 'Jonkheer','Col','Rev', 'Capt','Sir','Don'], ['Miss','Miss','Miss', 'Mr','Mr','Mrs','Mrs', 'Other','Other','Other', 'Mr','Mr','Mr']))}
    _df = df.withColumn('Initial',  F.regexp_extract( df['Name'], ('([A-Za-z]+)\.'),1 ) )
    _df = _df.replace(dizip_initials,1,'Initial')
    return _df

m = __import__("__main__")
setattr(m, 'InitialsTransformer', InitialsTransformer)
  
class MissingAgeTransformer(Transformer, DefaultParamsReadable, DefaultParamsWritable):
  @keyword_only
  def __init__(self):
    super(MissingAgeTransformer, self).__init__()

  def _transform(self, df: DataFrame) -> DataFrame:
    _df = df
    _df = _df.withColumn('Age', 
           F.when((F.isnull(_df['Age'])) & (_df['Initial'] == 'Mr') , 33 )\
            .otherwise(F.when((F.isnull(_df['Age'])) 
                              & (_df['Initial'] == 'Mrs') , 36)\
            .otherwise(F.when((F.isnull(_df['Age'])) 
                              & (_df['Initial'] == 'Master') , 5)\
            .otherwise(F.when((F.isnull(_df['Age'])) 
                              & (_df['Initial'] == 'Miss') , 22)\
            .otherwise(F.when((F.isnull(_df['Age'])) 
                              & (_df['Initial'] == 'Other') , 46)\
            .otherwise(_df['Age']) )))))
    return _df

setattr(m, 'MissingAgeTransformer', MissingAgeTransformer)
  
class FamilySizeTransformer(Transformer, DefaultParamsReadable, DefaultParamsWritable):
  @keyword_only
  def __init__(self):
    super(FamilySizeTransformer, self).__init__()

  def _transform(self, df: DataFrame) -> DataFrame:
    _df = df.withColumn('FamilySize', df['Parch'] + df['SibSp'] + 1 )
    return _df

setattr(m, 'FamilySizeTransformer', FamilySizeTransformer)
  
class IsAloneTransformer(Transformer, DefaultParamsReadable, DefaultParamsWritable):
  @keyword_only
  def __init__(self):
    super(IsAloneTransformer, self).__init__()

  def _transform(self, df: DataFrame) -> DataFrame:
    _df = df.withColumn('IsAlone', F.when(df['FamilySize'] > 1, 0).otherwise(1))
    return _df

setattr(m, 'IsAloneTransformer', IsAloneTransformer)
  
class FarePerPersonTransformer(Transformer, DefaultParamsReadable, DefaultParamsWritable):
  @keyword_only
  def __init__(self):
    super(FarePerPersonTransformer, self).__init__()

  def _transform(self, df: DataFrame) -> DataFrame:  
    _df = df.withColumn('FarePerPerson', df['Fare'] /df['FamilySize'])
    return _df

setattr(m, 'FarePerPersonTransformer', FarePerPersonTransformer)
  
class DropRowsWithNullTransformer(Transformer, DefaultParamsReadable, DefaultParamsWritable):
  def __init__(self): 
    super(DropRowsWithNullTransformer, self).__init__()
  
  col = Param(Params._dummy(), "col", "column", typeConverter=TypeConverters.toString)

  def setCol(self, value):
      return self._set(col=value)

  def getCol(self):
      return self.getOrDefault(self.col)

  def _transform(self, df: DataFrame) -> DataFrame:
    _df = df = df.filter(df[self.getCol()].isNotNull())
    return _df

setattr(m, 'DropRowsWithNullTransformer', DropRowsWithNullTransformer)
  
class CategoryToIndexTransformer(Transformer, DefaultParamsReadable, DefaultParamsWritable):
  def __init__(self): 
    super(CategoryToIndexTransformer, self).__init__()

  col = Param(Params._dummy(), "col", "column", typeConverter=TypeConverters.toString)

  def setCol(self, value):
      return self._set(col=value)

  def getCol(self):
      return self.getOrDefault(self.col)

  def _transform(self, df: DataFrame) -> DataFrame:
    indexer = StringIndexer(inputCol=self.getCol(), outputCol='{0}_indexed'.format(self.getCol()))
    _df = indexer.fit(df).transform(df)
    return _df

setattr(m, 'CategoryToIndexTransformer', CategoryToIndexTransformer)
  
class CategoryToOneHotEncodedTransformer(Transformer, DefaultParamsReadable, DefaultParamsWritable):
  def __init__(self): 
    super(CategoryToOneHotEncodedTransformer, self).__init__()

  cols = Param(Params._dummy(), "cols", "columns", typeConverter=TypeConverters.toListString)

  def setCols(self, value):
      return self._set(cols=value)

  def getCols(self):
      return self.getOrDefault(self.cols)

  def _transform(self, df: DataFrame) -> DataFrame:
    for col in self.getCols():
      cat_to_index = CategoryToIndexTransformer()
      cat_to_index.setCol(col)
      df = cat_to_index.transform(df)
    col_indexed = ['{0}_indexed'.format(col) for col in self.getCols()]
    col_encoded = ['{0}_encoded'.format(col) for col in self.getCols()]
    encoder = OneHotEncoderEstimator(inputCols=col_indexed, outputCols=col_encoded)
    _df = encoder.fit(df).transform(df)
    return _df

setattr(m, 'CategoryToOneHotEncodedTransformer', CategoryToOneHotEncodedTransformer)
  
class DropStringColsTransformer(Transformer, DefaultParamsReadable, DefaultParamsWritable):
  def __init__(self): 
    super(DropStringColsTransformer, self).__init__()

  cols = Param(Params._dummy(), "cols", "columns", typeConverter=TypeConverters.toListString)

  def setCols(self, value):
      return self._set(cols=value)

  def getCols(self):
      return self.getOrDefault(self.cols)
    
  def _transform(self, df: DataFrame) -> DataFrame:
    df = df.drop(*self.getCols())
    return df

setattr(m, 'DropStringColsTransformer', DropStringColsTransformer)

<>:11: DeprecationWarning: invalid escape sequence \.
<>:11: DeprecationWarning: invalid escape sequence \.
<command-3057669862061282>:11: DeprecationWarning: invalid escape sequence \.
 _df = df.withColumn('Initial', F.regexp_extract( df['Name'], ('([A-Za-z]+)\.'),1 ) )

In [15]:
data = spark.read.csv("wasbs://data@sparkmltrainig.blob.core.windows.net/train.csv", inferSchema = True, header = True)

train_df, test_df = data.randomSplit([0.9, 0.1], seed=42)


In [16]:
stages = []
init_trans = InitialsTransformer()
missing_age_trans = MissingAgeTransformer()
family_size_trans = FamilySizeTransformer()
is_alone_trans = IsAloneTransformer()
fare_per_person_trans = FarePerPersonTransformer()
drop_rows_with_null_trans = DropRowsWithNullTransformer()
drop_rows_with_null_trans.setCol('Embarked')
cat_to_one_how_trans = CategoryToOneHotEncodedTransformer()
cat_to_one_how_trans.setCols(['Sex', 'Initial', 'Embarked'])
drop_trans = DropStringColsTransformer()
drop_trans.setCols(['PassengerId', 'Cabin', 'Ticket', 'Name', 'Sex', 'Initial', 'Embarked'])

cols = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'FamilySize', 'IsAlone', 'FarePerPerson', 'Sex_indexed', 'Initial_indexed', 'Embarked_indexed', 'Sex_encoded', 'Initial_encoded', 'Embarked_encoded']
#cols = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'FamilySize', 'IsAlone', 'FarePerPerson']
vec_assembler = VectorAssembler(inputCols=cols, outputCol="Features")
lr = LinearRegression(featuresCol="Features", labelCol="Survived")

stages = [init_trans, family_size_trans, is_alone_trans, fare_per_person_trans, missing_age_trans, drop_rows_with_null_trans, cat_to_one_how_trans, drop_trans, vec_assembler, lr]
#stages = [init_trans, family_size_trans, is_alone_trans, fare_per_person_trans, missing_age_trans, drop_rows_with_null_trans, vec_assembler, lr]

pipeline = Pipeline(stages=stages)

pipelineModel = pipeline.fit(train_df)

In [17]:
pipelinePath = "/tmp/lr_pipeline_model"
pipelineModel.write().overwrite().save(pipelinePath)

In [18]:

#stages = [init_trans, family_size_trans, is_alone_trans, fare_per_person_trans, missing_age_trans, drop_rows_with_null_trans, cat_to_one_how_trans, drop_trans, vec_assembler]

#test_df_transformed = vec_assembler.transform(drop_trans.transform(cat_to_one_how_trans.transform(drop_rows_with_null_trans.transform(missing_age_trans.transform(fare_per_person_trans.transform(is_alone_trans.transform(family_size_trans.transform(init_trans.transform(test_df)))))))))

In [19]:
test_df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+-----------------+-------+-----+--------+
PassengerId|Survived|Pclass| Name| Sex| Age|SibSp|Parch| Ticket| Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+-----------------+-------+-----+--------+
 3| 1| 3|Heikkinen, Miss. ...|female|26.0| 0| 0| STON/O2. 3101282| 7.925| null| S|
 17| 0| 3|Rice, Master. Eugene| male| 2.0| 4| 1| 382652| 29.125| null| Q|
 36| 0| 1|Holverson, Mr. Al...| male|42.0| 1| 0| 113789| 52.0| null| S|
 51| 0| 3|Panula, Master. J...| male| 7.0| 4| 1| 3101295|39.6875| null| S|
 52| 0| 3|Nosworthy, Mr. Ri...| male|21.0| 0| 0| A/4. 39886| 7.8| null| S|
 100| 0| 2| Kantor, Mr. Sinai| male|34.0| 1| 0| 244367| 26.0| null| S|
 139| 0| 3| Osen, Mr. Olaf Elon| male|16.0| 0| 0| 7534| 9.2167| null| S|
 142| 1| 3|Nysten, Miss. Ann...|female|22.0| 0| 0| 347081| 7.75| null| S|
 174| 0| 3|Sivola, Mr. Antti...| male|21.0| 0| 0|STON/O 2. 3101280| 7.925| null| S|
 195| 1| 1|Brown, Mrs. James...|female|44.0| 0| 0| PC 17610|27.7208| B4| C|
 234| 1| 3|Asplund, Miss. Li...|female| 5.0| 4| 2| 347077|31.3875| null| S|
 235| 0| 2|Leyson, Mr. Rober...| male|24.0| 0| 0| C.A. 29566| 10.5| null| S|
 256| 1| 3|Touma, Mrs. Darwi...|female|29.0| 0| 2| 2650|15.2458| null| C|
 261| 0| 3| Smith, Mr. Thomas| male|null| 0| 0| 384461| 7.75| null| Q|
 266| 0| 2| Reeves, Mr. David| male|36.0| 0| 0| C.A. 17248| 10.5| null| S|
 272| 1| 3|Tornquist, Mr. Wi...| male|25.0| 0| 0| LINE| 0.0| null| S|
 279| 0| 3| Rice, Master. Eric| male| 7.0| 4| 1| 382652| 29.125| null| Q|
 291| 1| 1|"Barber, Miss. El...|female|26.0| 0| 0| 19877| 78.85| null| S|
 294| 0| 3| Haas, Miss. Aloisia|female|24.0| 0| 0| 349236| 8.85| null| S|
 321| 0| 3| Dennis, Mr. Samuel| male|22.0| 0| 0| A/5 21172| 7.25| null| S|
+-----------+--------+------+--------------------+------+----+-----+-----+-----------------+-------+-----+--------+
only showing top 20 rows

In [20]:
savedPipelineModel = PipelineModel.load(pipelinePath)
pred_df = savedPipelineModel.transform(test_df)


In [21]:
regression_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="Survived", metricName="rmse")

rmse = regression_evaluator.evaluate(pred_df)
print(f"RMSE is {rmse}")

RMSE is 0.34149375724898745